# **Signal To Spike Conversion** - Analysis

This notebook conducts an analysis of the Inter-Spike Interval in the UP and DN spike trains to determine the optimal synaptic time constants

### Check WD (change if necessary) and file loading

In [29]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/hfo/signal_to_spike" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/hfo/signal_to_spike"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

/home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/signal_to_spike


### Declare the `INPUT_FOLDER` and `RESULTS_FOLDER`

In [30]:
# CAREFUL WITH THIS FOLDER TO NOT OVERWRITE THE FILES
DATASET_FILENAME = "seeg_filtered_subset_90-119_segment500_200"
INPUT_FOLDER = f"results/{DATASET_FILENAME}"
RESULTS_FOLDER = f"analysis/{DATASET_FILENAME}"

## Load the UP and DN spike trains in the Ripple and FR Band

In [31]:
import numpy as np
import math
from utils.io import preview_np_array
from utils.input import read_spike_events, MarkerType, band_to_file_name, BaselineAlgorithm

### Load the Baseline Thresholds from the output file from the baseline process

In [32]:
# Specify the chosen Baseline Algorithm
chosen_baseline_alg_suffix = BaselineAlgorithm.MEDIAN

In [33]:
# Load the Baseline Thresholds
BASELINE_FILE = f"baseline_results/{DATASET_FILENAME}_thresholds_{chosen_baseline_alg_suffix}.npy"
baseline_thresholds = np.load(BASELINE_FILE)

# preview_np_array(baseline_thresholds, "baseline_thresholds", edge_items=3)

baseline_ripple_thresh = round(baseline_thresholds[0], 4)
baseline_fr_thresh = round(baseline_thresholds[1], 4)

# For now, the UP and DN thresholds are the same (symmetric)
ripple_thresh_up = baseline_ripple_thresh
ripple_thresh_down = -baseline_ripple_thresh
fr_thresh_up = baseline_fr_thresh
fr_thresh_down = -baseline_fr_thresh

print("Ripple Thresholds: ", ripple_thresh_up, ripple_thresh_down)
print("FR Thresholds: ", fr_thresh_up, fr_thresh_down)

Ripple Thresholds:  2.3602 -2.3602
FR Thresholds:  1.2731 -1.2731


In [34]:
ripple_band_filename = band_to_file_name(MarkerType.RIPPLE)
fr_band_filename = band_to_file_name(MarkerType.FAST_RIPPLE)

# Call the function to read the spike events
ripple_up_spikes_file_path = f"{INPUT_FOLDER}/{ripple_band_filename}_up_spike_train_{ripple_thresh_up}.csv"
ripple_down_spikes_file_path = f"{INPUT_FOLDER}/{ripple_band_filename}_down_spike_train_{ripple_thresh_down}.csv"

ripple_up_spike_train = read_spike_events(ripple_up_spikes_file_path)
ripple_down_spike_train = read_spike_events(ripple_down_spikes_file_path)


preview_np_array(ripple_up_spike_train, "ripple_up_spike_train", edge_items=3)
preview_np_array(ripple_down_spike_train, "ripple_down_spike_train", edge_items=3)

ripple_up_spike_train Shape: (7788, 2).
Preview: [[ 4.24804688e+01 -1.00000000e+00]
 [ 5.17578125e+01 -1.00000000e+00]
 [ 5.90820312e+01 -1.00000000e+00]
 ...
 [ 1.19975586e+05 -1.00000000e+00]
 [ 1.19989258e+05 -1.00000000e+00]
 [ 1.19998047e+05 -1.00000000e+00]]
ripple_down_spike_train Shape: (7830, 2).
Preview: [[ 4.44335938e+01 -1.00000000e+00]
 [ 5.46875000e+01 -1.00000000e+00]
 [ 5.66406250e+01 -1.00000000e+00]
 ...
 [ 1.19957520e+05 -1.00000000e+00]
 [ 1.19978027e+05 -1.00000000e+00]
 [ 1.19992188e+05 -1.00000000e+00]]


In [35]:
fr_up_spikes_file_path = f"{INPUT_FOLDER}/{fr_band_filename}_up_spike_train_{fr_thresh_up}.csv"
fr_down_spikes_file_path = f"{INPUT_FOLDER}/{fr_band_filename}_down_spike_train_{fr_thresh_down}.csv"

fr_up_spike_train = read_spike_events(fr_up_spikes_file_path)
fr_down_spike_train = read_spike_events(fr_down_spikes_file_path)

preview_np_array(fr_up_spike_train, "fr_up_spike_train", edge_items=3)
preview_np_array(fr_down_spike_train, "fr_down_spike_train", edge_items=3)

fr_up_spike_train Shape: (12917, 2).
Preview: [[ 1.86523438e+02 -1.00000000e+00]
 [ 1.93359375e+02 -1.00000000e+00]
 [ 2.01171875e+02 -1.00000000e+00]
 ...
 [ 1.19975586e+05 -1.00000000e+00]
 [ 1.19978027e+05 -1.00000000e+00]
 [ 1.19981445e+05 -1.00000000e+00]]
fr_down_spike_train Shape: (12853, 2).
Preview: [[ 1.87500000e+02 -1.00000000e+00]
 [ 1.94335938e+02 -1.00000000e+00]
 [ 2.02148438e+02 -1.00000000e+00]
 ...
 [ 1.19959961e+05 -1.00000000e+00]
 [ 1.19976562e+05 -1.00000000e+00]
 [ 1.19980469e+05 -1.00000000e+00]]


### Load the Annotated Data
The annotated data is needed to separate the intervals of HFO from the normal activity to calculate the Inter-Spike Interval (ISI)

In [36]:
# load the npy object
ripple_ground_truth = np.load(f"{INPUT_FOLDER}/{ripple_band_filename}_ground_truth.npy")
fr_ground_truth = np.load(f"{INPUT_FOLDER}/{fr_band_filename}_ground_truth.npy")

preview_np_array(ripple_ground_truth, "ripple_ground_truth", edge_items=3)
preview_np_array(fr_ground_truth, "fr_ground_truth", edge_items=3)

ripple_ground_truth Shape: (222,).
Preview: [('Fast-Ripple',   1000.  , 0.)
 ('Spike+Ripple+Fast-Ripple',   3206.54, 0.)
 ('Spike+Ripple',   3521.  , 0.) ... ('Spike+Ripple', 116216.  , 0.)
 ('Ripple+Fast-Ripple', 116769.  , 0.) ('Ripple', 119000.  , 0.)]
fr_ground_truth Shape: (199,).
Preview: [('Fast-Ripple',   1000.  , 0.)
 ('Spike+Ripple+Fast-Ripple',   3206.54, 0.)
 ('Fast-Ripple',   3770.02, 0.) ... ('Fast-Ripple', 116096.  , 0.)
 ('Ripple+Fast-Ripple', 116769.  , 0.) ('Fast-Ripple', 119000.  , 0.)]


## Iterate the UP and DN Spike Trains to Calculate the Inter-Spike Interval (ISI) in the HFO and Normal Activity

Define the lists to store:
- `Ripple_HFO_UP` - ISI of the UP train in the Ripple band during an HFO event 
- `Ripple_HFO_DN` - ISI of the DN train in the Ripple band during an HFO event
- `Ripple_Noise_UP` - ISI of the UP train in the Ripple band during normal activity
- `Ripple_Noise_DN` - ISI of the DN train in the Ripple band during normal activity

- `FR_HFO_UP` - ISI of the UP train in the Fast Ripple band during an HFO event
- `FR_HFO_DN` - ISI of the DN train in the Fast Ripple band during an HFO event
- `FR_Noise_UP` - ISI of the UP train in the Fast Ripple band during normal activity
- `FR_Noise_DN` - ISI of the DN train in the Fast Ripple band during normal activity

## Let's analyze the Ripple Band first

In [37]:
isi_ripple_hfo_up = []
isi_ripple_hfo_down = []
isi_ripple_noise_up = []
isi_ripple_noise_down = []

#### Ripple Band - UP Train

In [38]:
from utils.input import RIPPLE_CONFIDENCE_WINDOW, FR_CONFIDENCE_WINDOW, BOTH_CONFIDENCE_WINDOW

# Iterate the Ripple UP Spike Train
curr_ripple_gt_idx = 0  # Current index of the ripple ground truth
prev_spike_time = ripple_up_spike_train[0][0]

# Track the first spike inside an HFO
is_first_spike = True
for spike_idx in range(1, len(ripple_up_spike_train), 1):
    spike_time = ripple_up_spike_train[spike_idx][0]
    isi = spike_time - prev_spike_time
    # print("spike_time: ", spike_time, "prev_spike_time: ", prev_spike_time, "isi: ", isi)

    # Update the previous spike time
    prev_spike_time = spike_time

    # Check if the current spike time is within the ripple ground truth
    # Skip the ground truth that is before the current spike time
    while curr_ripple_gt_idx < len(ripple_ground_truth) and spike_time > ripple_ground_truth[curr_ripple_gt_idx][1] + RIPPLE_CONFIDENCE_WINDOW:
        curr_ripple_gt_idx += 1

    # Check if the current spike time is within the ripple ground truth
    if curr_ripple_gt_idx < len(ripple_ground_truth):
        curr_gt_start = ripple_ground_truth[curr_ripple_gt_idx][1]
        if spike_time >= curr_gt_start and spike_time <= curr_gt_start + RIPPLE_CONFIDENCE_WINDOW:
            # The spike is within the ripple ground truth

            # different_hfo verifies if the current spike is in a different HFO
            different_hfo = isi > RIPPLE_CONFIDENCE_WINDOW

            print(f"Spike time: {spike_time}. ISI: {isi}. is_first_spike: {is_first_spike}. different_hfo: {different_hfo}")
            if is_first_spike or different_hfo:
                # The first Spike inside an HFO is still considered as noise ISI since the prev_spike_time is outside the HFO
                isi_ripple_noise_up.append(isi)
                is_first_spike = False
            else:
                # Consider the ISI inside the HFO
                isi_ripple_hfo_up.append(isi)

            # Go to the next spike
            continue
    
    # The spike is not within the ripple ground truth
    isi_ripple_noise_up.append(isi)
    # Reset the first spike flag
    is_first_spike = True

Spike time: 1009.765625. ISI: 16.11328125. is_first_spike: True. different_hfo: False
Spike time: 1013.18359375. ISI: 3.41796875. is_first_spike: False. different_hfo: False
Spike time: 1017.578125. ISI: 4.39453125. is_first_spike: False. different_hfo: False
Spike time: 1021.484375. ISI: 3.90625. is_first_spike: False. different_hfo: False
Spike time: 1052.734375. ISI: 31.25. is_first_spike: False. different_hfo: False
Spike time: 1065.91796875. ISI: 13.18359375. is_first_spike: False. different_hfo: False
Spike time: 1078.61328125. ISI: 12.6953125. is_first_spike: False. different_hfo: False
Spike time: 3208.0078125. ISI: 9.27734375. is_first_spike: True. different_hfo: False
Spike time: 3209.47265625. ISI: 1.46484375. is_first_spike: False. different_hfo: False
Spike time: 3220.21484375. ISI: 10.7421875. is_first_spike: False. different_hfo: False
Spike time: 3220.703125. ISI: 0.48828125. is_first_spike: False. different_hfo: False
Spike time: 3221.19140625. ISI: 0.48828125. is_firs

In [39]:
print(f"Max ISI Ripple UP HFO: {max(isi_ripple_hfo_up)}")
print(f"Min ISI Ripple UP HFO: {min(isi_ripple_hfo_up)}")

Max ISI Ripple UP HFO: 91.796875
Min ISI Ripple UP HFO: 0.48828125


#### Ripple Band - DOWN Train

In [40]:
# Iterate the Ripple DOWN Spike Train
curr_ripple_gt_idx = 0  # Current index of the ripple ground truth
prev_spike_time = ripple_down_spike_train[0][0]

# Track the first spike inside an HFO
is_first_spike = True
for spike_idx in range(1, len(ripple_down_spike_train), 1):
    spike_time = ripple_down_spike_train[spike_idx][0]
    isi = spike_time - prev_spike_time
    # print("spike_time: ", spike_time, "prev_spike_time: ", prev_spike_time, "isi: ", isi)

    # Update the previous spike time
    prev_spike_time = spike_time

    # Check if the current spike time is within the ripple ground truth
    # Skip the ground truth that is before the current spike time
    while curr_ripple_gt_idx < len(ripple_ground_truth) and spike_time > ripple_ground_truth[curr_ripple_gt_idx][1] + RIPPLE_CONFIDENCE_WINDOW:
        curr_ripple_gt_idx += 1
    
    # Check if the current spike time is within the ripple ground truth
    if curr_ripple_gt_idx < len(ripple_ground_truth):
        curr_gt_start = ripple_ground_truth[curr_ripple_gt_idx][1]
        if spike_time >= curr_gt_start and spike_time <= curr_gt_start + RIPPLE_CONFIDENCE_WINDOW:
            # The spike is within the ripple ground truth

            # different_hfo verifies if the current spike is in a different HFO
            different_hfo = isi > RIPPLE_CONFIDENCE_WINDOW
            if is_first_spike or different_hfo:
                # The first Spike inside an HFO is still considered as noise ISI since the prev_spike_time is outside the HFO
                isi_ripple_noise_down.append(isi)
                is_first_spike = False
            else:
                # Consider the ISI inside the HFO
                isi_ripple_hfo_down.append(isi)
            # Go to the next spike
            continue
    
    # The spike is not within the ripple ground truth
    isi_ripple_noise_down.append(isi)
    # Reset the first spike flag
    is_first_spike = True

In [41]:
print(f"Max ISI Ripple DOWN HFO: {max(isi_ripple_hfo_down)}")
print(f"Min ISI Ripple DOWN HFO: {min(isi_ripple_hfo_down)}")

Max ISI Ripple DOWN HFO: 87.890625
Min ISI Ripple DOWN HFO: 0.48828125


### Validate the number of calculated ISIs
Validate that:
- `len(isi_ripple_hfo_up)` + `len(isi_ripple_noise_up)` = `len(ripple_up)` - 1 (first spike does not have an ISI)
- `len(isi_ripple_hfo_down_)` + `len(isi_ripple_noise_down_)` = `len(ripple_down_)` - 1 (first spike does not have an ISI)

In [42]:
#  Validate that len(isi_ripple_hfo_up) + len(isi_ripple_noise_up) = len(ripple_up)
print(f"Length isi_ripple_hfo_up: {len(isi_ripple_hfo_up)}")
print(f"Length isi_ripple_noise_up: {len(isi_ripple_noise_up)}")
print(f"Sum: {len(isi_ripple_hfo_up) + len(isi_ripple_noise_up)}")
print(f"Length ripple_up: {len(ripple_up_spike_train)}")

Length isi_ripple_hfo_up: 4164
Length isi_ripple_noise_up: 3623
Sum: 7787
Length ripple_up: 7788


In [43]:
#  Validate that len(isi_ripple_hfo_down) + len(isi_ripple_noise_down) = len(ripple_down)
print(f"Length isi_ripple_hfo_down: {len(isi_ripple_hfo_down)}")
print(f"Length isi_ripple_noise_down: {len(isi_ripple_noise_down)}")
print(f"Sum: {len(isi_ripple_hfo_down) + len(isi_ripple_noise_down)}")
print(f"Length ripple_down: {len(ripple_down_spike_train)}")

Length isi_ripple_hfo_down: 4146
Length isi_ripple_noise_down: 3683
Sum: 7829
Length ripple_down: 7830


## Let's analyze the Fast Ripple Band next

In [44]:
isi_fr_hfo_up = []
isi_fr_hfo_down = []
isi_fr_noise_up = []
isi_fr_noise_down = []

#### Fast Ripple Band - UP Train

In [45]:
# Iterate the FR UP Spike Train
curr_fr_gt_idx = 0  # Current index of the fast ripple ground truth
prev_spike_time = fr_up_spike_train[0][0]

# Track the first spike inside an HFO
is_first_spike = True

for spike_idx in range(1, len(fr_up_spike_train), 1):
    spike_time = fr_up_spike_train[spike_idx][0]
    isi = spike_time - prev_spike_time
    # print("spike_time: ", spike_time, "prev_spike_time: ", prev_spike_time, "isi: ", isi)

    # Update the previous spike time
    prev_spike_time = spike_time

    # Check if the current spike time is within the fast ripple ground truth
    # Skip the ground truth that is before the current spike time
    while curr_fr_gt_idx < len(fr_ground_truth) and spike_time > fr_ground_truth[curr_fr_gt_idx][1] + FR_CONFIDENCE_WINDOW:
        curr_fr_gt_idx += 1
    
    # Check if the current spike time is within the fast ripple ground truth
    if curr_fr_gt_idx < len(fr_ground_truth):
        curr_gt_start = fr_ground_truth[curr_fr_gt_idx][1]
        if spike_time >= curr_gt_start and spike_time <= curr_gt_start + FR_CONFIDENCE_WINDOW:
            # The spike is within the fast ripple ground truth

            # different_hfo verifies if the current spike is in a different HFO
            different_hfo = isi > FR_CONFIDENCE_WINDOW
            if is_first_spike or different_hfo:
                # The first Spike inside an HFO is still considered as noise ISI since the prev_spike_time is outside the HFO
                isi_fr_noise_up.append(isi)
                is_first_spike = False
            else:
                # Consider the ISI inside the HFO
                isi_fr_hfo_up.append(isi)

            # Go to the next spike
            continue
    
    # The spike is not within the fast ripple ground truth
    isi_fr_noise_up.append(isi)
    # Reset the first spike flag
    is_first_spike = True

#### Fast Ripple Band - DN Train

In [46]:
# Iterate the FR UP Spike Train
curr_fr_gt_idx = 0  # Current index of the fast ripple ground truth
prev_spike_time = fr_down_spike_train[0][0]

# Track the first spike inside an HFO
is_first_spike = True

for spike_idx in range(1, len(fr_down_spike_train), 1):
    spike_time = fr_down_spike_train[spike_idx][0]
    isi = spike_time - prev_spike_time
    # print("spike_time: ", spike_time, "prev_spike_time: ", prev_spike_time, "isi: ", isi)

    # Update the previous spike time
    prev_spike_time = spike_time

    # Check if the current spike time is within the fast ripple ground truth
    # Skip the ground truth that is before the current spike time
    while curr_fr_gt_idx < len(fr_ground_truth) and spike_time > fr_ground_truth[curr_fr_gt_idx][1] + FR_CONFIDENCE_WINDOW:
        curr_fr_gt_idx += 1
    
    # Check if the current spike time is within the fast ripple ground truth
    if curr_fr_gt_idx < len(fr_ground_truth):
        curr_gt_start = fr_ground_truth[curr_fr_gt_idx][1]
        if spike_time >= curr_gt_start and spike_time <= curr_gt_start + FR_CONFIDENCE_WINDOW:
            # The spike is within the fast ripple ground truth
            
            # different_hfo verifies if the current spike is in a different HFO
            different_hfo = isi > FR_CONFIDENCE_WINDOW
            if is_first_spike or different_hfo:
                # The first Spike inside an HFO is still considered as noise ISI since the prev_spike_time is outside the HFO
                isi_fr_noise_down.append(isi)
                is_first_spike = False
            else:
                # Consider the ISI inside the HFO
                isi_fr_hfo_down.append(isi)

            # Go to the next spike
            continue
    
    # The spike is not within the fast ripple ground truth
    isi_fr_noise_down.append(isi)
    # Reset the first spike flag
    is_first_spike = True

### Validate the number of calculated ISIs
Validate that:
- `len(isi_fr_hfo_up)` + `len(isi_fr_noise_up)` = `len(fr_up)` - 1 (first spike does not have an ISI)
- `len(isi_fr_hfo_down)` + `len(isi_fr_noise_down)` = `len(fr_down)` - 1 (first spike does not have an ISI)

In [47]:
#  Validate that len(isi_fr_hfo_up) + len(isi_fr_noise_up) = len(ripple_up)
print(f"Length isi_fr_hfo_up: {len(isi_fr_hfo_up)}")
print(f"Length isi_fr_noise_up: {len(isi_fr_noise_up)}")
print(f"Sum: {len(isi_fr_hfo_up) + len(isi_fr_noise_up)}")
print(f"Length fr_up: {len(fr_up_spike_train)}")

Length isi_fr_hfo_up: 3629
Length isi_fr_noise_up: 9287
Sum: 12916
Length fr_up: 12917


In [48]:
#  Validate that len(isi_fr_hfo_down) + len(isi_fr_noise_down) = len(ripple_down)
print(f"Length isi_fr_hfo_down: {len(isi_fr_hfo_down)}")
print(f"Length isi_fr_noise_down: {len(isi_fr_noise_down)}")
print(f"Sum: {len(isi_fr_hfo_down) + len(isi_fr_noise_down)}")
print(f"Length fr_down: {len(fr_down_spike_train)}")

Length isi_fr_hfo_down: 3635
Length isi_fr_noise_down: 9217
Sum: 12852
Length fr_down: 12853


--- 

## Calculate metrics of the ISIs and Show the Results

In [49]:
# Ripple band ISIs
# UP
# ---- Relevant Event Metrics ----
mean_ripple_hfo_up = np.mean(isi_ripple_hfo_up)
median_ripple_hfo_up = np.median(isi_ripple_hfo_up)
std_ripple_hfo_up = np.std(isi_ripple_hfo_up)
# Calculate the IQR for the ISI in the Ripple Band for the UP spikes during HFO
q1_ripple_hfo_up = np.percentile(isi_ripple_hfo_up, 25)
q3_ripple_hfo_up = np.percentile(isi_ripple_hfo_up, 75)
iqr_ripple_hfo_up = q3_ripple_hfo_up - q1_ripple_hfo_up

# ---- Baseline Activity Metrics ----
mean_ripple_noise_up = np.mean(isi_ripple_noise_up)
median_ripple_noise_up = np.median(isi_ripple_noise_up)
std_ripple_noise_up = np.std(isi_ripple_noise_up)
# Calculate the IQR for the ISI in the Ripple Band for the UP spikes during Noise
q1_ripple_noise_up = np.percentile(isi_ripple_noise_up, 25)
q3_ripple_noise_up = np.percentile(isi_ripple_noise_up, 75)
iqr_ripple_noise_up = q3_ripple_noise_up - q1_ripple_noise_up

# DOWN
# ---- Relevant Event Metrics ----
mean_ripple_hfo_down = np.mean(isi_ripple_hfo_down)
median_ripple_hfo_down = np.median(isi_ripple_hfo_down)
std_ripple_hfo_down = np.std(isi_ripple_hfo_down)
# Calculate the IQR for the ISI in the Ripple Band for the DOWN spikes during HFO
q1_ripple_hfo_down = np.percentile(isi_ripple_hfo_down, 25)
q3_ripple_hfo_down = np.percentile(isi_ripple_hfo_down, 75)
iqr_ripple_hfo_down = q3_ripple_hfo_down - q1_ripple_hfo_down

# ---- Baseline Activity Metrics ----
mean_ripple_noise_down = np.mean(isi_ripple_noise_down)
median_ripple_noise_down = np.median(isi_ripple_noise_down)
std_ripple_noise_down = np.std(isi_ripple_noise_down)
# Calculate the IQR for the ISI in the Ripple Band for the DOWN spikes during Noise
q1_ripple_noise_down = np.percentile(isi_ripple_noise_down, 25)
q3_ripple_noise_down = np.percentile(isi_ripple_noise_down, 75)
iqr_ripple_noise_down = q3_ripple_noise_down - q1_ripple_noise_down

In [50]:
# Fast Ripple band ISIs
# UP
# ---- Relevant Event Metrics ----
mean_fr_hfo_up = np.mean(isi_fr_hfo_up)
median_fr_hfo_up = np.median(isi_fr_hfo_up)
std_fr_hfo_up = np.std(isi_fr_hfo_up)
# Calculate the IQR for the ISI in the Fast Ripple Band for the UP spikes during HFO
q1_fr_hfo_up = np.percentile(isi_fr_hfo_up, 25)
q3_fr_hfo_up = np.percentile(isi_fr_hfo_up, 75)
iqr_fr_hfo_up = q3_fr_hfo_up - q1_fr_hfo_up

# ---- Baseline Activity Metrics ----
mean_fr_noise_up = np.mean(isi_fr_noise_up)
median_fr_noise_up = np.median(isi_fr_noise_up)
std_fr_noise_up = np.std(isi_fr_noise_up)
# Calculate the IQR for the ISI in the Fast Ripple Band for the UP spikes during Noise
q1_fr_noise_up = np.percentile(isi_fr_noise_up, 25)
q3_fr_noise_up = np.percentile(isi_fr_noise_up, 75)
iqr_fr_noise_up = q3_fr_noise_up - q1_fr_noise_up

# DOWN
# ---- Relevant Event Metrics ----
mean_fr_hfo_down = np.mean(isi_fr_hfo_down)
median_fr_hfo_down = np.median(isi_fr_hfo_down)
std_fr_hfo_down = np.std(isi_fr_hfo_down)
# Calculate the IQR for the ISI in the Fast Ripple Band for the DOWN spikes during HFO
q1_fr_hfo_down = np.percentile(isi_fr_hfo_down, 25)
q3_fr_hfo_down = np.percentile(isi_fr_hfo_down, 75)
iqr_fr_hfo_down = q3_fr_hfo_down - q1_fr_hfo_down

# ---- Baseline Activity Metrics ----
mean_fr_noise_down = np.mean(isi_fr_noise_down)
median_fr_noise_down = np.median(isi_fr_noise_down)
std_fr_noise_down = np.std(isi_fr_noise_down)
# Calculate the IQR for the ISI in the Fast Ripple Band for the DOWN spikes during Noise
q1_fr_noise_down = np.percentile(isi_fr_noise_down, 25)
q3_fr_noise_down = np.percentile(isi_fr_noise_down, 75)
iqr_fr_noise_down = q3_fr_noise_down - q1_fr_noise_down

### Print the Metrics of the ISIs

#### Ripple Band

In [58]:
# Metrics of the ISIs in the Ripple Band
print("Ripple UP")
print("HFO ISI (ms)")
print(f"Mean: {mean_ripple_hfo_up}, Median: {median_ripple_hfo_up}, STD: {std_ripple_hfo_up}, IQR: [{q1_ripple_hfo_up} - {q3_ripple_hfo_up}], Max: {max(isi_ripple_hfo_up)}, Min: {min(isi_ripple_hfo_up)}")
print("Noise ISI (ms)")
print(f"Mean: {mean_ripple_noise_up}, Median: {median_ripple_noise_up}, STD: {std_ripple_noise_up}, IQR: [{q1_ripple_noise_up} - {q3_ripple_noise_up}], Max: {max(isi_ripple_noise_up)}, Min: {min(isi_ripple_noise_up)}")

print("=================================")
print("Ripple DOWN")
print("HFO ISI (ms)")
print(f"Mean: {mean_ripple_hfo_down}, Median: {median_ripple_hfo_down}, STD: {std_ripple_hfo_down}, IQR: [{q1_ripple_hfo_down} - {q3_ripple_hfo_down}], Max: {max(isi_ripple_hfo_down)}, Min: {min(isi_ripple_hfo_down)}")
print("Noise ISI (ms)")
print(f"Mean: {mean_ripple_noise_down}, Median: {median_ripple_noise_down}, STD: {std_ripple_noise_down}, IQR: [{q1_ripple_noise_down}  - {q3_ripple_noise_down}], Max: {max(isi_ripple_noise_down)}, Min: {min(isi_ripple_noise_down)}")

Ripple UP
HFO ISI (ms)
Mean: 4.033479864613352, Median: 0.9765625, STD: 8.102743156258079, IQR: [0.48828125 - 4.8828125], Max: 91.796875, Min: 0.48828125
Noise ISI (ms)
Mean: 28.473683756555342, Median: 11.71875, STD: 49.666969913412764, IQR: [7.8125 - 28.3203125], Max: 605.46875, Min: 0.48828125
Ripple DOWN
HFO ISI (ms)
Mean: 3.9193226528581766, Median: 0.9765625, STD: 7.762278371432685, IQR: [0.48828125 - 4.8828125], Max: 87.890625, Min: 0.48828125
Noise ISI (ms)
Mean: 28.155916966467554, Median: 11.71875, STD: 49.38114504541526, IQR: [7.8125  - 28.3203125], Max: 598.6328125, Min: 0.48828125


#### Fast Ripple Band

In [59]:
# Metrics of the ISIs in the FR Band
print("FR UP")
print("HFO ISI (ms)")
print(f"Mean: {mean_fr_hfo_up}, Median: {median_fr_hfo_up}, STD: {std_fr_hfo_up}, IQR: [{q1_fr_hfo_up} - {q3_fr_hfo_up}], Max: {max(isi_fr_hfo_up)}, Min: {min(isi_fr_hfo_up)}")
print("Noise ISI (ms)")
print(f"Mean: {mean_fr_noise_up}, Median: {median_fr_noise_up}, STD: {std_fr_noise_up}, IQR: [{q1_fr_noise_up} - {q3_fr_noise_up}], Max: {max(isi_fr_noise_up)}, Min: {min(isi_fr_noise_up)}")

print("=================================")
print("FR DOWN")
print("HFO ISI (ms)")
print(f"Mean: {mean_fr_hfo_down}, Median: {median_fr_hfo_down}, STD: {std_fr_hfo_down}, IQR: [{q1_fr_hfo_down} - {q3_fr_hfo_down}], Max: {max(isi_fr_hfo_down)}, Min: {min(isi_fr_hfo_down)}")
print("Noise ISI (ms)")
print(f"Mean: {mean_fr_noise_down}, Median: {median_fr_noise_down}, STD: {std_fr_noise_down}, IQR: [{q1_fr_noise_down} - {q3_fr_noise_down}], Max: {max(isi_fr_noise_down)}, Min: {min(isi_fr_noise_down)}")

FR UP
HFO ISI (ms)
Mean: 2.2580148629098926, Median: 0.48828125, STD: 3.564652795751104, IQR: [0.48828125 - 2.9296875], Max: 47.36328125, Min: 0.48828125
Noise ISI (ms)
Mean: 12.016860766393885, Median: 3.90625, STD: 27.858294045325724, IQR: [2.44140625 - 12.20703125], Max: 552.734375, Min: 0.48828125
FR DOWN
HFO ISI (ms)
Mean: 2.2771234525447044, Median: 0.9765625, STD: 3.606039539947184, IQR: [0.48828125 - 2.9296875], Max: 48.33984375, Min: 0.48828125
Noise ISI (ms)
Mean: 12.098906911142453, Median: 3.90625, STD: 27.950719897285705, IQR: [2.44140625 - 12.20703125], Max: 552.734375, Min: 0.48828125


### Display Box Plots with the Inter-Spike Intervals

#### Ripple ISIs

In [53]:
# Display Box Plots with the Inter-Spike Intervals
from utils.bar_plot import create_box_plot
import bokeh.plotting as bplt

# Ripple Band Box Plot
# Define the array of values
ripple_isi_list = [isi_ripple_hfo_up, isi_ripple_hfo_down, isi_ripple_noise_up, isi_ripple_noise_down]
# Create the BoxPlot
ripple_isi_boxplot = create_box_plot(
    title="Inter-Spike Interval of the UP/DOWN Spike trains in the Ripple Band", 
    box_arrays=ripple_isi_list,
    y_axis_label='Inter-Spike Interval (ms)',
    x_axis_labels=['HFO UP', 'HFO DOWN', 'NOISE UP', 'NOISE DOWN'],
    sizing_mode="stretch_width",
)
# TODO: Add legend on the x-axis

show_ripple_isi_barplot = True
if show_ripple_isi_barplot:
    # Show the plot
    bplt.show(ripple_isi_boxplot)

ripple_quantiles:  [0.48828125 0.9765625  4.8828125 ]
Ripple IQR:  4.39453125
ripple_quantiles:  [0.48828125 0.9765625  4.8828125 ]
Ripple IQR:  4.39453125
ripple_quantiles:  [ 7.8125    11.71875   28.3203125]
Ripple IQR:  20.5078125
ripple_quantiles:  [ 7.8125    11.71875   28.3203125]
Ripple IQR:  20.5078125


In [54]:
# Export the plot to an HTML file
EXPORT_RIPPLE_BOXPLOT = True
if EXPORT_RIPPLE_BOXPLOT:
    # Create the folder if it does not exist
    if not os.path.exists(RESULTS_FOLDER):
        os.makedirs(RESULTS_FOLDER)

    ripple_boxplot_filename = f"{RESULTS_FOLDER}/ripple_isi_boxplot_thresh{ripple_thresh_up}-{ripple_thresh_down}.html"

    # Customize the output file settings
    bplt.output_file(filename=ripple_boxplot_filename, title="Inter-Spike Interval of the UP/DOWN Spike trains in the Ripple Band")

    # Save the plot
    bplt.save(ripple_isi_boxplot)

    # Close the plot
    bplt.reset_output()

#### Fast Ripple ISIs

In [55]:
# Display Box Plots with the Inter-Spike Intervals

# FR Band Box Plot
# Define the array of values
fr_isi_list = [isi_fr_hfo_up, isi_fr_hfo_down, isi_fr_noise_up, isi_fr_noise_down]
# Create the BoxPlot
fr_isi_boxplot = create_box_plot(
    title="Inter-Spike Interval of the UP/DOWN Spike trains in the Fast Ripple Band", 
    box_arrays=fr_isi_list,
    y_axis_label='Inter-Spike Interval (ms)',
    x_axis_labels=['HFO UP', 'HFO DOWN', 'NOISE UP', 'NOISE DOWN'],
    sizing_mode="stretch_width",
)
# TODO: Add legend on the x-axis

show_fr_isi_barplot = True
if show_fr_isi_barplot:
    # Show the plot
    bplt.show(fr_isi_boxplot)

ripple_quantiles:  [0.48828125 0.48828125 2.9296875 ]
Ripple IQR:  2.44140625
ripple_quantiles:  [0.48828125 0.9765625  2.9296875 ]
Ripple IQR:  2.44140625
ripple_quantiles:  [ 2.44140625  3.90625    12.20703125]
Ripple IQR:  9.765625
ripple_quantiles:  [ 2.44140625  3.90625    12.20703125]
Ripple IQR:  9.765625


In [56]:
# Export the plot to an HTML file
EXPORT_FR_PLOT = True
if EXPORT_FR_PLOT:
    # Create the folder if it does not exist
    if not os.path.exists(RESULTS_FOLDER):
        os.makedirs(RESULTS_FOLDER)

    fr_boxplot_filename = f"{RESULTS_FOLDER}/fr_isi_boxplot_thresh{fr_thresh_up}-{fr_thresh_down}.html"

    # Customize the output file settings
    bplt.output_file(filename=fr_boxplot_filename, title="Inter-Spike Interval of the UP/DOWN Spike trains in the Fast Ripple Band")

    # Save the plot
    bplt.save(fr_isi_boxplot)

    bplt.reset_output()